# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [35]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [36]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chulman,56.8479,124.9104,-12.33,100,100,1.83,RU,1675621225
1,1,bluff,-46.6000,168.3333,59.40,87,94,33.62,NZ,1675621151
2,2,nikolskoye,59.7035,30.7861,22.91,83,100,7.36,RU,1675620993
3,3,castro,-24.7911,-50.0119,75.00,64,78,5.37,BR,1675621227
4,4,prince rupert,54.3161,-130.3201,40.69,79,100,8.05,CA,1675621006


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot=city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles="OSM",
    size="Humidity",
    color="City")

# Display the map 
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Narrow down cities that fit criteria and drop any results with null values
vacay_cities=city_data_df.loc[(city_data_df["Max Temp"]>70)&(city_data_df["Wind Speed"]<5)].dropna(how='any')

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
vacay_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,pueblo bello,8.2041,-76.5260,86.50,53,64,3.87,CO,1675621231
18,18,ugoofaaru,5.6667,73.0000,80.76,75,100,4.25,MV,1675621233
22,22,puerto quijarro,-17.7833,-57.7667,86.76,54,99,4.56,BO,1675621234
38,38,pringsewu,-5.3582,104.9744,73.92,79,100,1.88,ID,1675621240
39,39,alta floresta,-9.8756,-56.0861,87.62,66,40,2.30,BR,1675621241
...,...,...,...,...,...,...,...,...,...,...
523,523,tshikapa,-6.4167,20.8000,71.11,89,24,4.52,CD,1675621439
525,525,banda aceh,5.5577,95.3222,77.99,80,98,2.80,ID,1675621439
529,529,kawalu,-7.3817,108.2082,73.35,88,100,1.39,ID,1675621441
539,539,pontianak,-0.0333,109.3333,76.98,94,75,0.00,ID,1675621251


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=vacay_cities[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
13,pueblo bello,CO,8.2041,-76.5260,53,
18,ugoofaaru,MV,5.6667,73.0000,75,
22,puerto quijarro,BO,-17.7833,-57.7667,54,
38,pringsewu,ID,-5.3582,104.9744,79,
39,alta floresta,BR,-9.8756,-56.0861,66,
...,...,...,...,...,...,...
523,tshikapa,CD,-6.4167,20.8000,89,
525,banda aceh,ID,5.5577,95.3222,80,
529,kawalu,ID,-7.3817,108.2082,88,
539,pontianak,ID,-0.0333,109.3333,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":20,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=row['Lat']
    lng=row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params).json()
    
    # Convert the API response to JSON format
    #name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pueblo bello - nearest hotel: No hotel found
ugoofaaru - nearest hotel: No hotel found
puerto quijarro - nearest hotel: No hotel found
pringsewu - nearest hotel: No hotel found
alta floresta - nearest hotel: Villas Hotel
kieta - nearest hotel: No hotel found
bundaberg - nearest hotel: Young Australian Hotel
miri - nearest hotel: Kenyalang Suites Miri
sao miguel do araguaia - nearest hotel: Hotel São Miguel
mahebourg - nearest hotel: Grand Bel Air
nakhon thai - nearest hotel: Chantuk Resort
boyolangu - nearest hotel: Hotel Puspita
tonj - nearest hotel: No hotel found
letlhakane - nearest hotel: Boteti Hotel
inirida - nearest hotel: Residencias El Turista del Guainia
padang - nearest hotel: losmen Surya
diamantino - nearest hotel: Hotel kayaby
cayenne - nearest hotel: Central Hôtel
necochea - nearest hotel: Cabañas Río Quequén
nsanje - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
beni - nearest hotel: Logement Alco-Family
ambon - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
13,pueblo bello,CO,8.2041,-76.5260,53,No hotel found
18,ugoofaaru,MV,5.6667,73.0000,75,No hotel found
22,puerto quijarro,BO,-17.7833,-57.7667,54,No hotel found
38,pringsewu,ID,-5.3582,104.9744,79,No hotel found
39,alta floresta,BR,-9.8756,-56.0861,66,Villas Hotel
...,...,...,...,...,...,...
523,tshikapa,CD,-6.4167,20.8000,89,Hôtel TSHAMUNTU
525,banda aceh,ID,5.5577,95.3222,80,Hotel 61
529,kawalu,ID,-7.3817,108.2082,88,Hotel Surya
539,pontianak,ID,-0.0333,109.3333,94,Golden Tulip Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [46]:
hotel_df=hotel_df[hotel_df["Hotel Name"]!="No hotel found"]
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
39,alta floresta,BR,-9.8756,-56.0861,66,Villas Hotel
59,bundaberg,AU,-24.8500,152.3500,97,Young Australian Hotel
82,miri,MY,4.4148,114.0089,100,Kenyalang Suites Miri
84,sao miguel do araguaia,BR,-13.2750,-50.1628,63,Hotel São Miguel
90,mahebourg,MU,-20.4081,57.7000,89,Grand Bel Air
101,nakhon thai,TH,17.1006,100.8374,78,Chantuk Resort
121,boyolangu,ID,-8.1181,111.8935,82,Hotel Puspita
145,letlhakane,BW,-21.4167,25.5833,42,Boteti Hotel
170,inirida,CO,3.8653,-67.9239,62,Residencias El Turista del Guainia
175,padang,ID,-0.9492,100.3543,79,losmen Surya


In [48]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
prettymap=hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles="EsriNatGeo",
    frame_width=700,
    frame_height=500,
    scale=.1,
    color="City",
    hover_cols=["Country","Hotel Name"]
    )

# Display the map
prettymap

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)